<a href="https://colab.research.google.com/github/FernandoBRdgz/inteligencia_artificial/blob/main/incrustaciones_de_palabras/word2vec_yelp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Introducción

El conjunto de datos de Yelp es un subconjunto de nuestros negocios, reseñas y datos de usuario para su uso con fines personales, educativos y académicos. Disponible como archivos JSON, úselo para enseñar a los estudiantes acerca de las bases de datos, para aprender NLP o para obtener datos de producción de muestra mientras aprende a crear aplicaciones móviles.

Enlace al conjunto de datos: https://www.kaggle.com/datasets/yelp-dataset/yelp-dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import json
from pprint import pprint

In [3]:
main_path = '/content/drive/MyDrive/'

In [4]:
data_directory = os.path.join(main_path, 'data', 'yelp_dataset')

In [5]:
businesses_filepath = os.path.join(data_directory, 'yelp_academic_dataset_business.json')

In [6]:
with open(businesses_filepath) as f:
    first_business_record = f.readline() 

pprint(first_business_record)

('{"business_id":"Pns2l4eNsfO8kk83dixA6A","name":"Abby Rappoport, LAC, '
 'CMQ","address":"1616 Chapala St, Ste 2","city":"Santa '
 'Barbara","state":"CA","postal_code":"93101","latitude":34.4266787,"longitude":-119.7111968,"stars":5.0,"review_count":7,"is_open":0,"attributes":{"ByAppointmentOnly":"True"},"categories":"Doctors, '
 'Traditional Chinese Medicine, Naturopathic\\/Holistic, Acupuncture, Health & '
 'Medical, Nutritionists","hours":null}\n')


In [7]:
review_json_filepath = os.path.join(data_directory, 'yelp_academic_dataset_review.json')

In [8]:
with open(review_json_filepath) as f:
    first_review_record = f.readline()
    
pprint(first_review_record)

('{"review_id":"KU_O5udG6zpxOg-VcAEodg","user_id":"mh_-eMZ6K5RLWhZyISBhwA","business_id":"XQfwVwDr-v0ZS3_CbbE5Xw","stars":3.0,"useful":0,"funny":0,"cool":0,"text":"If '
 'you decide to eat here, just be aware it is going to take about 2 hours from '
 'beginning to end. We have tried it multiple times, because I want to like '
 "it! I have been to it's other locations in NJ and never had a bad "
 'experience. \\n\\nThe food is good, but it takes a very long time to come '
 'out. The waitstaff is very young, but usually pleasant. We have just had too '
 'many experiences where we spent way too long waiting. We usually opt for '
 'another diner or restaurant on the weekends, in order to be done '
 'quicker.","date":"2018-07-07 22:09:11"}\n')


In [9]:
restaurant_ids = set()

with open(businesses_filepath) as f:    
    for business_json in f:
        business = json.loads(business_json)
        if not business.get('categories'):
            continue
        if 'Restaurants' not in business['categories']:
            continue
        restaurant_ids.add(business['business_id'])

restaurant_ids = frozenset(restaurant_ids)

pprint(f'{len(restaurant_ids):,} restaurants in the dataset.')

'52,268 restaurants in the dataset.'


In [10]:
scratch_directory = os.path.join(data_directory, 'scratch')

try:
    os.mkdir(scratch_directory)
except FileExistsError:
    pass

review_txt_filepath = os.path.join(scratch_directory, 'review_text_all.txt')

In [11]:
%%time
execute = False

if execute:
    review_count = 0
    with open(review_txt_filepath, 'w') as review_txt_file:
        with open(review_json_filepath) as review_json_file:
            for review_json in review_json_file:
                review = json.loads(review_json)
                if review['business_id'] not in restaurant_ids:
                    continue
                review_txt_file.write(review['text'].replace('\n', '\\n') + '\n')
                review_count += 1
    print(f'Text from {review_count:,} restaurant reviews written to the new txt file.')
    
else:
    with open(review_txt_filepath) as review_txt_file:
        for review_count, line in enumerate(review_txt_file):
            pass
        
    print(f'Text from {review_count + 1:,} restaurant reviews in the txt file.')

Text from 4,725,884 restaurant reviews in the txt file.
CPU times: user 9.51 s, sys: 1.78 s, total: 11.3 s
Wall time: 40.1 s


In [12]:
import spacy
from spacy import displacy
import pandas as pd
import itertools as it

In [13]:
!python -m spacy download en_core_web_md

2023-05-17 21:49:41.832794: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 10.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [14]:
nlp = spacy.load('en_core_web_md')

In [15]:
review_num = 42

with open(review_txt_filepath) as f:
    sample_review = list(it.islice(f, review_num, review_num+1))[0]
    sample_review = sample_review.replace('\\n', '\n')
        
print(sample_review)

Excellent service! Great diner food and breakfast is served all day. Came here for lunch- they were busy but very friendly and hospitable. Easy to get to off the 295.



In [16]:
%%time
parsed_review = nlp(sample_review)

CPU times: user 26.9 ms, sys: 817 µs, total: 27.7 ms
Wall time: 50.9 ms


In [17]:
print(parsed_review)

Excellent service! Great diner food and breakfast is served all day. Came here for lunch- they were busy but very friendly and hospitable. Easy to get to off the 295.



In [18]:
displacy.render(parsed_review, style="ent", jupyter=True)

In [19]:
from gensim.models.phrases import Phrases, Phraser
from gensim.models.word2vec import LineSentence

In [20]:
def punct_space(token):
    return token.is_punct or token.is_space

def pronoun_lemmatize(token):
    if token.lemma_ == '-PRON-':
        return token.lower_
    
    else:
        return token.lemma_.lower()

def line_review(filename):
    with open(filename) as f:
        for review in f:
            yield review.replace('\\n', '\n')

In [21]:
review_lemmatized_filepath = os.path.join(scratch_directory, 'review_lemmatized_all.txt')
sentence_lemmatized_filepath = os.path.join(scratch_directory, 'sentence_lemmatized_all.txt')

In [22]:
%%time
execute = False

if execute:
    with open(review_lemmatized_filepath, 'w') as review_file:
        with open(sentence_lemmatized_filepath, 'w') as sentence_file:
            pipe = nlp.pipe(
                line_review(review_txt_filepath),
                batch_size=5000
                )
            
            for parsed_review in pipe:
                lemmatized_review = ' '.join([
                    pronoun_lemmatize(token)
                    for token in parsed_review
                    if not punct_space(token)
                    ])
                
                review_file.write(lemmatized_review + '\n')
        
                for sent in parsed_review.sents:
                    lemmatized_sentence = ' '.join([
                        pronoun_lemmatize(token)
                        for token in sent
                        if not punct_space(token)
                        ])
                    
                    sentence_file.write(lemmatized_sentence + '\n')

CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 10.7 µs


In [23]:
sentences_unigrams = LineSentence(sentence_lemmatized_filepath)

In [24]:
for sentence_unigrams in it.islice(sentences_unigrams, 60, 70):
    print(' '.join(sentence_unigrams))
    print('')

the bun make the sonoran dog

it be like a snuggie for the pup

a first it seem ridiculous and almost like it be go to be too much exactly like everyone 's favorite blanket with sleeve

too much softness too much smush too indulgent

wrong

it be warm soft chewy fragrant and it succeed where other famed sonoran dogs fail

the hot dog itself be flavorful but i would prefer that it or the bacon have a little more bite or snap to well hold their own against the dominant mustard and onion

i be with the masse on the carne asada caramelo

excellent tortilla salty melty cheese and great carne

super cheap and you can drive through



In [25]:
bigram_model_filepath = os.path.join(scratch_directory, 'bigram_phrase_model')

In [26]:
%%time
execute = False

if execute:

    bigram_phrases = Phrases(sentences_unigrams)
    bigram_phrases = Phraser(bigram_phrases)
    bigram_phrases.save(bigram_model_filepath)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.34 µs


In [27]:
bigram_phrases = Phraser.load(bigram_model_filepath)

In [28]:
sentences_bigrams_filepath = os.path.join(scratch_directory, 'sentence_bigram_phrases_all.txt')

In [29]:
%%time
execute = False
if execute:
    with open(sentences_bigrams_filepath, 'w') as f:
        for sentence_unigrams in sentences_unigrams:
            sentence_bigrams = ' '.join(bigram_phrases[sentence_unigrams])
            f.write(sentence_bigrams + '\n')

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.82 µs


In [30]:
sentences_bigrams = LineSentence(sentences_bigrams_filepath)

In [31]:
for sentence_bigrams in it.islice(sentences_bigrams, 60, 70):
    print(' '.join(sentence_bigrams))
    print('')

the bun make the sonoran_dog

it be like a snuggie for the pup

a first it seem ridiculous and almost like it be go to be too much exactly like everyone 's favorite blanket with sleeve

too much softness too much smush too indulgent

wrong

it be warm soft chewy fragrant and it succeed where other famed sonoran_dogs fail

the hot_dog itself be flavorful but i would prefer that it or the bacon have a little more bite or snap to well hold their own against the dominant mustard and onion

i be with the masse on the carne_asada caramelo

excellent tortilla salty melty_cheese and great carne

super cheap and you can drive_through



In [32]:
trigram_model_filepath = os.path.join(scratch_directory, 'trigram_phrase_model')

In [33]:
%%time
execute = False

if execute:

    trigram_phrases = Phrases(sentences_bigrams)
    trigram_phrases = Phraser(trigram_phrases)
    trigram_phrases.save(trigram_model_filepath)

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 9.06 µs


In [34]:
trigram_phrases = Phraser.load(trigram_model_filepath)

In [35]:
sentences_trigrams_filepath = os.path.join(scratch_directory, 'sentence_trigram_phrases_all.txt')

In [36]:
%%time
execute = False

if execute:
    with open(sentences_trigrams_filepath, 'w') as f:
        for sentence_bigrams in sentences_bigrams:
            sentence_trigrams = ' '.join(trigram_phrases[sentence_bigrams])
            f.write(sentence_trigrams + '\n')

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.82 µs


In [37]:
sentences_trigrams = LineSentence(sentences_trigrams_filepath)

In [38]:
for sentence_trigrams in it.islice(sentences_trigrams, 60, 70):
    print(' '.join(sentence_trigrams))
    print('')

the bun make the sonoran_dog

it be like a snuggie for the pup

a first it seem ridiculous and almost like it be go to be too much exactly like everyone 's favorite blanket with sleeve

too much softness too much smush too indulgent

wrong

it be warm soft chewy fragrant and it succeed where other famed sonoran_dogs fail

the hot_dog itself be flavorful but i would prefer that it or the bacon have a little more bite or snap to well hold their own against the dominant mustard and onion

i be with the masse on the carne_asada_caramelo

excellent tortilla salty melty_cheese and great carne

super cheap and you can drive_through



In [39]:
review_trigrams_filepath = os.path.join(scratch_directory, 'review_trigrams_all.txt')

In [40]:
%%time
execute = False

if execute:
    reviews_lemmatized = LineSentence(review_lemmatized_filepath)

    with open(review_trigrams_filepath, 'w') as f:
        
        for review_unigrams in reviews_lemmatized:
            review_bigrams = bigram_phrases[review_unigrams]
            review_trigrams = trigram_phrases[review_bigrams]

            review_trigrams = [
                term
                for term in review_trigrams
                if term not in nlp.Defaults.stop_words
                ]

            review_trigrams = ' '.join(review_trigrams)
            f.write(review_trigrams + '\n')

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.78 µs


In [41]:
review_num = 0

print('Original:' + '\n')

for review in it.islice(line_review(review_txt_filepath), review_num, review_num+1):
    print(review)

print('----' + '\n')
print('Transformed:' + '\n')

with open(review_trigrams_filepath) as f:
    for review in it.islice(f, review_num, review_num+1):
        print(review)

Original:

If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. We have tried it multiple times, because I want to like it! I have been to it's other locations in NJ and never had a bad experience. 

The food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant. We have just had too many experiences where we spent way too long waiting. We usually opt for another diner or restaurant on the weekends, in order to be done quicker.

----

Transformed:

decide eat aware 2 hour begin end try multiple time want like location nj bad experience food good long time come waitstaff young usually pleasant experience spend way long wait usually opt diner restaurant weekend order quick



In [42]:
!pip install pyLDAvis==2.1.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97721 sha256=0b913e309233d0207df8a1bab5029f7f2d28a0385eb053c19e1ca39ca16517bc
  Stored in directory: /root/.cache/pip/wheels/d9/93/d6/16c95da19c32f037fd75135ea152d0df37254c25cd1a8b4b6c
Successfully built pyLDAvis


In [43]:
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis
import pyLDAvis.gensim
import warnings
import pickle

In [49]:
pyLDAvis.enable_notebook()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [44]:
dictionary_filepath = os.path.join(scratch_directory, 'trigram_dict_all.dict')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [50]:
%%time
execute = False

if execute:
    reviews_trigrams = LineSentence(review_trigrams_filepath)
    dictionary_trigrams = Dictionary(reviews_trigrams)
    dictionary_trigrams.filter_extremes(no_below=20, no_above=0.4)
    dictionary_trigrams.compactify()
    dictionary_trigrams.save(dictionary_filepath)  

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.3 µs


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [51]:
dictionary_trigrams = Dictionary.load(dictionary_filepath)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [52]:
bow_corpus_filepath = os.path.join(scratch_directory, 'bow_trigrams_corpus_all.mm')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [53]:
def bow_generator(filepath):
   
    for review in LineSentence(filepath):
        yield dictionary_trigrams.doc2bow(review)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [54]:
%%time
execute = False

if execute:
    MmCorpus.serialize(bow_corpus_filepath, bow_generator(review_trigrams_filepath))

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 10.3 µs


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [55]:
trigram_bow_corpus = MmCorpus(bow_corpus_filepath)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [56]:
lda_model_filepath = os.path.join(scratch_directory, 'lda_model_all')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [57]:
%%time
execute = False

if execute:
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        lda = LdaMulticore(trigram_bow_corpus, num_topics=50, id2word=dictionary_trigrams, workers=7)
    
    lda.save(lda_model_filepath)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.91 µs


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [58]:
lda = LdaMulticore.load(lda_model_filepath)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [59]:
def explore_topic(topic_number, topn=25):
    print(f'{"term":20} {"frequency"}' + '\n')

    for term, frequency in lda.show_topic(topic_number, topn=topn):
        print(f'{term:20} {round(frequency, 3):.3f}')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [70]:
explore_topic(topic_number=30, topn=5)

term                 frequency

sushi                0.069
roll                 0.051
place                0.023
great                0.015
fresh                0.013


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [71]:
topic_names = {
    0: 'place1',
    1: 'sauce',
    2: 'place2',
    3: 'time',
    4: 'service1',
    5: 'seafood1',
    6: 'reservation',
    7: 'taste',
    8: 'donut',
    9: 'vietnam',
    10: 'orders1',
    11: 'time',
    12: 'salad',
    13: 'order2',
    14: 'burgers & fries',
    15: 'mexican',
    16: 'order3',
    17: 'seafood2',
    18: 'staff',
    19: 'atmosphere',
    20: 'chip',
    21: 'restaurant',
    22: 'place',
    23: 'service2',
    24: 'bar',
    25: 'bagel',
    26: 'indian',
    27: 'vegan',
    28: 'pizza',
    29: 'bread',
    30: 'sushi',
    31: 'barbecue',
    32: 'local business',
    33: 'miscellaneous',
    34: 'hole-in-the-wall',
    35: 'asian',
    36: 'specials',
    37: 'coffeeshop',
    38: 'prices',
    39: 'flavor & texture',
    40: 'noodles',
    41: 'canadian',
    42: 'highly recommended',
    43: 'sushi',
    44: 'ordering',
    45: 'mediterranean',
    46: 'decent value',
    47: 'cleanliness',
    48: 'lobster',
    49: 'seafood'
    }

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [72]:
topic_names_filepath = os.path.join(scratch_directory, 'topic_names.pkl')

with open(topic_names_filepath, 'wb') as f:
    pickle.dump(topic_names, f)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [73]:
LDAvis_data_filepath = os.path.join(scratch_directory, 'ldavis_prepared')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [74]:
%%time
execute = False

if execute:

    LDAvis_prepared = pyLDAvis.gensim.prepare(lda, trigram_bow_corpus, dictionary_trigrams)

    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f) 

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/pyLDAvis/_prepare.py:232: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  head(R).drop('saliency', 1)


CPU times: user 5min 33s, sys: 1min 2s, total: 6min 36s
Wall time: 6min 3s


In [75]:
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [76]:
pyLDAvis.display(LDAvis_prepared)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**Por hacer**

* Añadir comentarios
* Incrustaciones de palabra con Word2vec
* Visualizaciones
* Álgebra de palabras

**Referencias**

* https://spacy.io/
* https://radimrehurek.com/gensim/
* https://github.com/pwharrison/modern-nlp-in-python-2019/blob/master/notebooks/Modern_NLP_in_Python.ipynb